In [1]:
import gradio as gr
import numpy as np
from PIL import Image
import cv2
from ultralytics import YOLO
from transformers import pipeline

In [3]:
yolo_model = YOLO("license-plate-detector/weights/best.pt")
dino_model = pipeline("zero-shot-object-detection", model="IDEA-Research/grounding-dino-base")

Device set to use cpu


In [4]:
def detect_with_yolo(image: Image.Image):
    results = yolo_model(np.array(image))[0]
    return Image.fromarray(results.plot())

def detect_with_grounding_dino(image: Image.Image, prompt="vehicle registration plate."):
    results = dino_model(image, candidate_labels=[prompt])
    image_np = np.array(image).copy()

    if not results:
        return Image.fromarray(image_np)

    results = [result for result in results if result["score"] > 0.4]

    for result in results:
        box = result["box"]
        score = result["score"]
        label = "license plate"

        x1, y1, x2, y2 = int(box["xmin"]), int(box["ymin"]), int(box["xmax"]), int(box["ymax"])
        image_np = cv2.rectangle(image_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
        image_np = cv2.putText(image_np, f"{label} ({score:.2f})", (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return Image.fromarray(image_np)

In [5]:
example_images = [
    ["example_images/2975157083_4567dde5d5_z.jpg"],
    ["example_images/3984649872_216bc62270_z.jpg"],
    ["example_images/6143184254_98522ca297_z.jpg"]
]

In [6]:
def process_image(image):
    yolo_out = detect_with_yolo(image)
    dino_out = detect_with_grounding_dino(image)
    return yolo_out, dino_out

In [8]:
gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Image(label="YOLOv8 Detection"),
        gr.Image(label="Grounding DINO Detection")
    ],
    examples=example_images,
    title="Kennzeichenerkennung",
    description="Lade ein Bild hoch oder wähle ein Beispielbild und vergleiche die Ergebnisse."
).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
